In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tidy3d as td
import tidy3d.web as web


# --- 1. Material Definitions ---
# Using fixed indices for 1550nm for simplicity

wavelength =  1.55                              # Central wavelength
wv_points = 50                   # number of wavelength wv_points
width_220 = 0.390
width_300 = 0.310
MMI_L_points = 10
MMI_W_points = 10
n_si = 3.475
n_sio2 = 1.458
MMI_L_optimized = 7.8
MMI_W_optimized = 3
taper_width = 0.8
Initial_pos = -11



sweep_wavelength = np.linspace(1.5,1.6,wv_points)  # Sweep on wavelengths
sweep_freq = td.C_0 / sweep_wavelength          # Sweep on Frequencies
sweep_MMI_L = np.linspace(MMI_L_optimized-0.5,MMI_L_optimized+0.5,MMI_L_points)
sweep_MMI_W = np.linspace(MMI_W_optimized-0.5,MMI_W_optimized+0.5,MMI_W_points)
dinstance_tapers = sweep_MMI_W/2



bandwidth = sweep_wavelength[-1]-sweep_wavelength[0]

mat_si = td.material_library["cSi"]["Li1993_293K"] # Material trace permitivity model for crystaline Silicon
mat_sio2 = td.material_library["SiO2"]["Horiba"]   # Material trace permitivity model for crystaline Silica


web.configure("U7IS5Q1AW3qYVxPAAQDE7vgxaTz6Kt6aI9AioPZUajVRQQ2o")

In [1]:
def build_MMI_simulation(
    MMI_length = np.array([MMI_L_optimized]),
    MMI_width = np.array([MMI_W_optimized]),
    width_single_mode=np.array([width_220,width_300]),
    thickness=np.array([0.22,0.3]),
    wavelength = np.array([1.55]),
):

    # Frequency
    freq = td.C_0 / wavelength

    # Materials
    core = mat_si
    clad = mat_sio2

    # --- We define the simulation data array and simulation objects for the two different sweeps----

    sim_data_arr = [[[]],[[]]] # Simulation data for 220nm and 300nm, Length sweep and Width sweep
    sim_arr = [[[]],[[]]]      # Simulation objects for 220nm and 300nm , Length sweep and Width sweep
    estimate = 0

    #--- Define Sources ---
    freq0 = td.C_0 / wavelength.mean()
    fwidth = (td.C_0 / wavelength.min()) - (td.C_0 / wavelength.max())



    for thick_idx,thick in enumerate(thickness):
        for length_idx,length_values in enumerate(MMI_length):
            for width_idx,width_values in enumerate(MMI_width):

                ## Definicion de la fuente ##
                source1 = td.ModeSource(
                    name = 'Mode_source',
                    center = [Initial_pos-3.5, 0, 0],
                    size = [0, 4, 4],
                    source_time = td.GaussianPulse(freq0 = freq0, fwidth = fwidth, ),
                    direction = '+',
                    mode_spec = td.ModeSpec(num_modes = 1, target_neff = n_si, sort_spec = {'filter_reference' : 0, 'filter_order':'over', 'sort_order':'ascending', 'track_freq':'central'}, group_index_step = True, ),
                )

                ## Definicion de los monitores ##

                Longitudinal = td.FieldMonitor(
                    name = 'Longitudinal',
                    size = [td.inf, 7.75, 0],
                    freqs = td.C_0 / np.linspace(1.5, 1.6, 10),
                )

                IN = td.FieldMonitor(
                    name = 'IN',
                    center = [Initial_pos-3.5, 0, 0],
                    size = [0, 4, 4],
                    freqs = td.C_0 / 1.55,
                )

                OUT1 = td.FieldMonitor(
                    name = 'OUT1',
                    center = [length_values + Initial_pos + 3, dinstance_tapers / 2, 0],
                    size = [0, 1, 1],
                    freqs = td.C_0 / 1.55,
                )

                OUT2 = td.FieldMonitor(
                    name = 'OUT2',
                    center = [length_values + Initial_pos + 3, -dinstance_tapers / 2, 0],
                    size = [0, 1, 1],
                    freqs = td.C_0 / 1.55,
                )


                ## Definicion de las estructuras ##


                ## for a Strip

                Xmin_Strip= (-100)
                Xmax_Strip= (-2+Initial_pos)

                Ymin_Strip= (-width_single_mode[thick_idx]/2)
                Ymax_Strip= (width_single_mode[thick_idx]/2)

                Zmin_Strip= (-thick/2)
                Zmax_Strip= (thick/2)

                Strip = td.Structure(
                    geometry = td.Box(center = [(Xmax_Strip+Xmin_Strip)/2, (Ymax_Strip+Ymin_Strip)/2, (Zmax_Strip+Zmin_Strip)/2], size = [Xmax_Strip-Xmin_Strip, Ymax_Strip-Ymin_Strip, Zmax_Strip-Zmin_Strip], ),
                    name = 'Strip',
                    medium = core
                )



                 ## For the MMI

                Xmin_MMI= Initial_pos
                Xmax_MMI= MMI_L+Initial_pos

                Ymin_MMI= -MMI_W/2
                Ymax_MMI= MMI_W/2

                Zmin_MMI= (-thick/2)
                Zmax_MMI= (thick/2)

                MMI = td.Structure(
                    geometry = td.Box(center = [(Xmax_MMI+Xmin_MMI)/2, (Ymax_MMI+Ymin_MMI)/2, (Zmax_MMI+Zmin_MMI)/2], size = [Xmax_MMI-Xmin_MMI, Ymax_MMI-Ymin_MMI, Zmax_MMI-Zmin_MMI]),
                    name = 'MMI',
                    medium = core
                )


                ## For the Taper In

                Xv1_Taper_IN =-2+Initial_pos
                Yv1_Taper_IN =-width_single_mode[thick_idx]/2

                Xv2_Taper_IN = Initial_pos
                Yv2_Taper_IN =-taper_width/2

                Xv3_Taper_IN = Initial_pos
                Yv3_Taper_IN = taper_width/2

                Xv4_Taper_IN = -2 + Initial_pos
                Yv4_Taper_IN = width_single_mode[thick_idx] / 2

                Zmin_Taper_IN= (-thick/2)
                Zmax_Taper_IN= (thick/2)


                Taper_IN = td.Structure(
                    geometry = td.PolySlab(slab_bounds = [Zmin_Taper_IN, Zmax_Taper_IN], vertices = [[Xv1_Taper_IN, Yv1_Taper_IN], [Xv2_Taper_IN, Yv2_Taper_IN], [Xv3_Taper_IN, Yv3_Taper_IN], [Xv4_Taper_IN, Yv4_Taper_IN]]),
                    name = 'Taper',
                    medium = core
                )

                 ## For the Taper OUT 1

                Xv1_Taper_OUT_1 = length_values+Initial_pos+2
                Yv1_Taper_OUT_1 =-width_single_mode[thick_idx]/2-dinstance_tapers[width_idx]/2

                Xv2_Taper_OUT_1 = length_values+Initial_pos
                Yv2_Taper_OUT_1 =-dinstance_tapers[width_idx]/2-taper_width/2

                Xv3_Taper_OUT_1 = length_values+Initial_pos
                Yv3_Taper_OUT_1 = -dinstance_tapers[width_idx]/2+taper_width/2

                Xv4_Taper_OUT_1 = length_values+Initial_pos+2
                Yv4_Taper_OUT_1 = -dinstance_tapers[width_idx]/2+width_single_mode[thick_idx]/2

                Zmin_Taper_OUT_1= (-thick/2)
                Zmax_Taper_OUT_1= (thick/2)



                Taper_out1 = td.Structure(
                    geometry = td.PolySlab(slab_bounds = [Zmin_Taper_OUT_1, Zmax_Taper_OUT_1], vertices = [[Xv1_Taper_OUT_1, Yv1_Taper_OUT_1], [Xv2_Taper_OUT_1, Yv2_Taper_OUT_1], [Xv3_Taper_OUT_1, Yv3_Taper_OUT_1], [Xv4_Taper_OUT_1, Yv4_Taper_OUT_1]]),
                    name = 'Taper_out1',
                    medium = core
                )


                ## For the Taper OUT 2

                Xv1_Taper_OUT_2 = length_values+Initial_pos+2
                Yv1_Taper_OUT_2 =width_single_mode[thick_idx]/2+dinstance_tapers[width_idx]/2

                Xv2_Taper_OUT_2 = length_values+Initial_pos
                Yv2_Taper_OUT_2 =dinstance_tapers[width_idx]/2+taper_width/2

                Xv3_Taper_OUT_2 = length_values+Initial_pos
                Yv3_Taper_OUT_2 = dinstance_tapers[width_idx]/2-taper_width/2

                Xv4_Taper_OUT_2 = length_values+Initial_pos+2
                Yv4_Taper_OUT_2 = dinstance_tapers[width_idx]/2-width_single_mode[thick_idx]/2

                Zmin_Taper_OUT_2= (-thick/2)
                Zmax_Taper_OUT_2= (thick/2)


                Taper_out2 = td.Structure(
                    geometry = td.PolySlab(slab_bounds = [Zmin_Taper_OUT_2, Zmax_Taper_OUT_2], vertices = [[Xv1_Taper_OUT_2, Yv1_Taper_OUT_2], [Xv2_Taper_OUT_2, Yv2_Taper_OUT_2], [Xv3_Taper_OUT_2, Yv3_Taper_OUT_2], [Xv4_Taper_OUT_2, Yv4_Taper_OUT_2]]),
                    name = 'Taper_out2',
                    medium = core
                )


                ## for a Strip_out1

                Xmin_Strip_out1= length_values+Initial_pos+2
                Xmax_Strip_out1= length_values+100

                Ymin_Strip_out1= -dinstance_tapers[width_idx]/2-width_single_mode[thick_idx]/2
                Ymax_Strip_out1= width_single_mode[thick_idx]/2-dinstance_tapers[width_idx]/2

                Zmin_Strip_out1= (-thick/2)
                Zmax_Strip_out1= (thick/2)



                Strip_out1 = td.Structure(
                    geometry = td.Box(center = [(Xmax_Strip_out1+Xmin_Strip_out1)/2, (Ymax_Strip_out1+Ymin_Strip_out1)/2, (Zmax_Strip_out1+Zmin_Strip_out1)/2], size = [Xmax_Strip_out1-Xmin_Strip_out1, Ymax_Strip_out1-Ymin_Strip_out1, Zmax_Strip_out1-Zmin_Strip_out1]),
                    name = 'Strip_out1',
                    medium = core
                )

                ## for a Strip_out2

                Xmin_Strip_out2= length_values+Initial_pos+2
                Xmax_Strip_out2= length_values+100

                Ymin_Strip_out2= dinstance_tapers[width_idx]/2-width_single_mode[thick_idx]/2
                Ymax_Strip_out2= width_single_mode[thick_idx]/2+dinstance_tapers[width_idx]/2

                Zmin_Strip_out2= (-thick/2)
                Zmax_Strip_out2= (thick/2)


                Strip_out2 = td.Structure(
                    geometry = td.Box(center = [(Xmax_Strip_out2+Xmin_Strip_out2)/2, (Ymax_Strip_out2+Ymin_Strip_out2)/2, (Zmax_Strip_out2+Zmin_Strip_out2)/2], size = [Xmax_Strip_out2-Xmin_Strip_out2, Ymax_Strip_out2-Ymin_Strip_out2, Zmax_Strip_out2-Zmin_Strip_out2]),
                    name = 'Strip_out2',
                    medium = core
                )



            # --- Simulation domain ---
            sim_arr[thick_idx][length_idx].append(td.Simulation(
                size=(15.5, 15.5, 15.5),  # x ignored for mode solver
                monitors=[OUT1,OUT2,Longitudinal,IN],
                sources=[source1],
                structures=[Strip,Strip_out1,Strip_out2,MMI,Taper_IN,Taper_out1,Taper_out2],
                medium=clad,
                symmetry=(0, 0, 1),  # symmetry in z
                run_time=1e-12,
                grid_spec=td.GridSpec.auto(wavelength = wavelength.min(),min_steps_per_wvl = 11),
            ))



            Job = web.Job(simulation=sim_arr[thick_idx][gap_idx], task_name="my_sim")

            estimate+= Job.estimate_cost()

            sim_data_arr[thick_idx].append(web.run(sim_arr[thick_idx][gap_idx], task_name='DC_FDTD'))
    print(f"Estimated Maximum Cost: {estimate}")

    return sim_data_arr, sim_arr


sim_data, sim = build_slot_simulation(gap = sweep_gap,wavelength=sweep_wavelength)

SyntaxError: invalid syntax (2101526925.py, line 121)